# Quantum Error Correction

In [1]:
from qiskit import QuantumCircuit
# noinspection PyUnresolvedReferences
from qiskit_aer import AerSimulator  # needed to make the qc.save_statevector available

# Setup Service and Backend

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
estimator = Estimator(backend)

# Circuit

In [3]:
from qiskit.quantum_info import Statevector


# See https://en.wikipedia.org/wiki/Five-qubit_error_correcting_code
def quantum_circuit_creator(state: Statevector):  # state must have 5 qubits
    qc = QuantumCircuit(10, 4)  # I use a 10th qubit to measure how much $\psi$ changed
    qc.prepare_state(state, [5, 6, 7, 8])
    qc.cx(0, 5)

    qc.cz(0, 6)
    # qc.ci(0,5) I don't think a Control Identity makes sense?

    qc.cz(0, 7)
    qc.cx(1, 6)
    qc.cx(2, 5)

    qc.cx(0, 8)
    qc.cz(1, 7)
    qc.cz(3, 5)

    qc.cz(1, 7)
    qc.cx(2, 6)
    qc.cx(3, 5)

    qc.cx(1, 8)
    qc.cz(2, 7)

    qc.cz(2, 8)
    qc.cx(3, 7)

    qc.cz(3, 8)

    qc.measure([5, 6, 7, 8], [0, 1, 2, 3])
    qc.barrier()
    qc.draw("mpl")
    
    return qc
    

qc = quantum_circuit_creator(state=Statevector.from_label("+-rl"))

In [4]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

n_qubits = 127

# mat = np.real(random_hermitian(n_qubits, seed=1234))
qc.measure_all()

In [5]:
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(qc)

# Setup Sampler

In [6]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(backend=backend)

# Run

In [11]:
job = sampler.run([isa_circuit])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [12]:
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]
print(f"Counts for the meas output register: {pub_result.data.meas.get_counts()}")

In [13]:
from qiskit.visualization import plot_histogram

plot_histogram(pub_result.data.meas.get_counts())

In [26]:
" | ".join([f"{v}: {k}" for k,v in sorted(pub_result.data.meas.get_counts().items(), key = lambda x: -x[1])])